In [121]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
import numpy as np
from numpy.linalg import norm
import clingo
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories1 import *

In [122]:
with open("config.yml","r") as config:
    data = load(config, Loader=Loader)
    api_key = data["apikey"]
    openai.api_key  = api_key

In [123]:
version = 1

In [124]:
projects = [sudoku,seeknumbers,minotaur,creek]

In [5]:
for i in range(len(projects)):
    query = dict(projects[i])
    example_1 = projects[i-2]
    example_2 = projects[i-3]
    story = query["story"]
    story_1 = example_1["story"]
    story_2 = example_2["story"]
    answer = pipeline(example_1,example_2,query)
    name_of_file = os.path.join("generated_solutions",story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version)+".lp")
    with open(name_of_file,"w") as file:
        file.write(answer)

In [125]:
def asp_try(asp_file, instance):
    solution = ""
    models = None
    success = 0
    try:
        ctl = Control(arguments=["--opt-mode=opt"])
        ctl.load(asp_file)
        ctl.load(instance)
        ctl.add("base", [], solution)
        parts = [("base", [])]
        ctl.ground(parts)      
        try:       
            with ctl.solve(yield_=True, async_=True) as handle: 
                handle.wait(10)
                handle.cancel()
                models = list(iter(handle))
                if len(models) > 0:
                    success = 1
                    #for m in models:
                    #    print(m.symbols(True,True,True))
            if ctl.statistics['solving']['conflicts'] > 0:
                print("Errors occurred during solving.")
                # Analyze the conflicts to get more information
                conflicts_analyzed = ctl.statistics['solving']['conflicts_analyzed']
                print("Conflicts analyzed: {}".format(conflicts_analyzed))
            else:
                print("No errors occurred during solving.")
        except:
            print("Unsatisfiable")
    except:
        print("Parsing or Safety Issues")
    return success, models

In [126]:
instances_dict ={"sudoku":os.path.join("projects","sudoku","instances","ex01.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","instances","ex01.lp"),
                 "minotaur":os.path.join("projects","minotaur","instances","level01.lp"),
                 "creek":os.path.join("projects","creek","instances","ex01.lp")
                }

In [8]:
for i in range(len(projects)):
    query = dict(projects[i])
    story = query["story"]
    story_1 = projects[i-1]["story"]
    story_2 = projects[i-2]["story"]
    name_of_file = os.path.join("generated_solutions",story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version)+".lp")
    print("="*40 + name_of_file + "="*40)
    with open(name_of_file,"r") as f:
        print("Length of File:" + str(len(f.readlines())))
    instance = instances_dict[story]
    success, models = asp_try(name_of_file,instance)
    print("Success: " + str(success))
    if models:
        for m in models:
            print(m.symbols(True,True,True))

========================================generated_solutions\sudoku_from_creek_and_minotaur_v1.lp========================================
Length of File:31
Parsing or Safety Issues
Success: 0
========================================generated_solutions\seeknumbers_from_sudoku_and_creek_v1.lp========================================
Length of File:12
Parsing or Safety Issues
Success: 0
========================================generated_solutions\minotaur_from_seeknumbers_and_sudoku_v1.lp========================================
Length of File:95
Parsing or Safety Issues
Success: 0
========================================generated_solutions\creek_from_minotaur_and_seeknumbers_v1.lp========================================
Length of File:40
Success: 0


### Now trying with the best matches

In [127]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [128]:
stories=[]
problems = []
embeddings = [] 
for project in projects:
    stories.append(project["story"])
    problems.append(project["problem"])

In [129]:
df = pd.DataFrame({"stories":stories,"problems":problems})
df.head(5)

,stories,problems
0,sudoku,The task of this project is to solve a Sudoku ...
1,seeknumbers,"Given a square grid as shown on the left side,..."
2,minotaur,The task of this project is to solve the Minot...
3,creek,The task of this project is to solve a Creek p...


In [130]:
df["embeddings"] = df.problems.apply(lambda x : get_embedding(x))

In [131]:
df.head()

,stories,problems,embeddings
0,sudoku,The task of this project is to solve a Sudoku ...,"[0.0061800191178917885, -0.0026702347677201033..."
1,seeknumbers,"Given a square grid as shown on the left side,...","[0.00273314630612731, -0.0024913938250392675, ..."
2,minotaur,The task of this project is to solve the Minot...,"[-0.008220619522035122, -0.0007007355452515185..."
3,creek,The task of this project is to solve a Creek p...,"[0.011148616671562195, 0.006013285368680954, 0..."


In [132]:
for project in projects:
    query = project["problem"]
    query_embedding = get_embedding(query)
    np_query = np.array(query_embedding)
    cosine_name = 'cosine_' + project['story']
    df[cosine_name] = df.embeddings.apply(lambda x : np.dot(np_query,np.array(x))/(norm(np_query)*norm(np.array(x))))

In [133]:
df.head()

,stories,problems,embeddings,cosine_sudoku,cosine_seeknumbers,cosine_minotaur,cosine_creek
0,sudoku,The task of this project is to solve a Sudoku ...,"[0.0061800191178917885, -0.0026702347677201033...",1.000000,0.837012,0.810473,0.896495
1,seeknumbers,"Given a square grid as shown on the left side,...","[0.00273314630612731, -0.0024913938250392675, ...",0.837127,0.999999,0.810117,0.893415
2,minotaur,The task of this project is to solve the Minot...,"[-0.008220619522035122, -0.0007007355452515185...",0.810554,0.810117,0.999999,0.832703
3,creek,The task of this project is to solve a Creek p...,"[0.011148616671562195, 0.006013285368680954, 0...",0.896495,0.893354,0.832629,1.000000


In [134]:
def get_best_context(df, projects, story, N = 2):
    column_cosine = 'cosine_' + story
    df_sorted = df.sort_values(by=column_cosine, ascending=False)
    second_best = df_sorted.iloc[1]  
    third_best = df_sorted.iloc[2] 
    best_stories = [second_best["stories"],third_best["stories"]]
    result = []
    for project in projects:
        if project["story"] in best_stories:
            result.append(project)
    return result

In [136]:
for project in projects:
    query = dict(project)
    example1, example_2 = get_best_context(df,projects,project["story"])
    story = query["story"]
    story_1 = example_1["story"]
    story_2 = example_2["story"]
    answer = pipeline(example_1,example_2,query)
    name_of_file = os.path.join("generated_solutions",story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version)+".lp")
    with open(name_of_file,"w") as file:
        file.write(answer)

In [137]:
for project in projects:
    example1, example_2 = get_best_context(df,projects,project["story"])
    story = project["story"]
    story_1 = example_1["story"]
    story_2 = example_2["story"]
    name_of_file = os.path.join("generated_solutions",story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version)+".lp")
    print("="*40 + name_of_file + "="*40)
    with open(name_of_file,"r") as f:
        print("Length of File:" + str(len(f.readlines())))
    instance = instances_dict[story]
    success, models = asp_try(name_of_file,instance)
    print("Success: " + str(success))
    if models:
        for m in models:
            print(m.symbols(True,True,True))

========================================generated_solutions\sudoku_from_seeknumbers_and_creek_v1.lp========================================
Length of File:26
Parsing or Safety Issues
Success: 0
========================================generated_solutions\seeknumbers_from_seeknumbers_and_creek_v1.lp========================================
Length of File:74
Parsing or Safety Issues
Success: 0
========================================generated_solutions\minotaur_from_seeknumbers_and_creek_v1.lp========================================
Length of File:41
Parsing or Safety Issues
Success: 0
========================================generated_solutions\creek_from_seeknumbers_and_seeknumbers_v1.lp========================================
Length of File:40
Parsing or Safety Issues
Success: 0


## The new project: hop

In [138]:
project = hop
example1, example_2 = get_best_context(df,projects,project["story"])
story = project["story"]
story_1 = example_1["story"]
story_2 = example_2["story"]
name_of_file = os.path.join("generated_solutions",story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version)+".lp")
print("="*40 + name_of_file + "="*40)
with open(name_of_file,"r") as f:
    print("Length of File:" + str(len(f.readlines())))
instance = instances_dict[story]
success, models = asp_try(name_of_file,instance)
print("Success: " + str(success))
if models:
    for m in models:
        print(m.symbols(True,True,True))

NameError: name 'hop' is not defined

### Another asp_try

In [72]:
asp_file = os.path.join("projects","sudoku","canonical_sudoku.lp")
generated = os.path.join("generated_solutions","minotaur_from_seeknumbers_and_sudoku_v2.lp")
instance = os.path.join("projects","sudoku","instances","ex00.lp")

In [73]:
def asp_try_v2(asp_file, instance):
    control = clingo.Control()
    input_files = [asp_file, instance]
    asp_program = []
    
    def on_model(model):
        print("Model:", model)
        
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
    control.add("base", [], "".join(asp_program))
    
    try:
        control.ground([("base", [])])
    except:
        print("Grounding Error")
        return None
        
    control.configuration.solve.models = 0  # Limit the number of models to 1
    try:
        control.solve(on_model=on_model)
        print("No errors occurred during solving.")
    except:
        #if control.statistics['solving']['solvers']['conflicts'] > 0:
        print("Errors occurred during solving.")
        # Analyze the conflicts to get more information
        conflicts_analyzed = control.statistics['solving'] #['conflicts_analyzed']
        print("Conflicts analyzed: {}".format(conflicts_analyzed))
        #else:            

In [74]:
asp_try_v2(generated, instance)

Grounding Error


In [33]:
ctl.statistics['solving']['solvers']

{'choices': 0.0,
 'conflicts': 0.0,
 'conflicts_analyzed': 0.0,
 'restarts': 0.0,
 'restarts_last': 0.0}